In [1]:
import pandas as pd
import numpy as np

from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
# 정답과 예측을 넣으면, 정확도를 산출해서 보여준다.
from sklearn.metrics import accuracy_score
# 평가지표를 한눈에 보게해주는 라이브러리
from sklearn.metrics import classification_report

In [2]:
baseData = pd.read_csv("../../dataset/keggledata.csv")
baseData

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [3]:
labelEn = LabelEncoder()

In [4]:
encodeData = baseData.copy()

encodeData["work_year2"] = labelEn.fit_transform(encodeData.work_year)
encodeData = encodeData.drop(columns=["work_year"])

encodeData["experience_level2"] = labelEn.fit_transform(encodeData.experience_level)
encodeData = encodeData.drop(columns=["experience_level"])

encodeData["employment_type2"] = labelEn.fit_transform(encodeData.employment_type)
encodeData = encodeData.drop(columns=["employment_type"])

encodeData["job_title2"] = labelEn.fit_transform(encodeData.job_title)
encodeData = encodeData.drop(columns=["job_title"])

encodeData["salary_currency2"] = labelEn.fit_transform(encodeData.salary_currency)
encodeData = encodeData.drop(columns=["salary_currency"])

encodeData["employee_residence2"] = labelEn.fit_transform(encodeData.employee_residence)
encodeData = encodeData.drop(columns=["employee_residence"])

encodeData["company_location2"] = labelEn.fit_transform(encodeData.company_location)
encodeData = encodeData.drop(columns=["company_location"])

encodeData["company_size2"] = labelEn.fit_transform(encodeData.company_size)
encodeData = encodeData.drop(columns=["company_size"])
encodeData

,salary,salary_in_usd,remote_ratio,work_year2,experience_level2,employment_type2,job_title2,salary_currency2,employee_residence2,company_location2,company_size2
0,80000,85847,100,3,3,2,84,7,26,25,0
1,30000,30000,100,3,2,0,66,19,75,70,2
2,25500,25500,100,3,2,0,66,19,75,70,2
3,175000,175000,100,3,3,2,47,19,11,12,1
4,120000,120000,100,3,3,2,47,19,11,12,1
...,...,...,...,...,...,...,...,...,...,...,...
3750,412000,412000,100,0,3,2,47,19,75,70,0
3751,151000,151000,100,1,2,2,84,19,75,70,0
3752,105000,105000,100,0,0,2,47,19,75,70,2
3753,100000,100000,100,0,0,0,17,19,75,70,0


In [5]:
corrDf = encodeData.corr()
stdRatio = 0.3
corrDf

,salary,salary_in_usd,remote_ratio,work_year2,experience_level2,employment_type2,job_title2,salary_currency2,employee_residence2,company_location2,company_size2
salary,1.000000,-0.023676,0.028731,-0.094724,-0.024884,-0.003908,0.039065,-0.091121,-0.087176,-0.072774,-0.100352
salary_in_usd,-0.023676,1.000000,-0.064171,0.228290,0.327173,-0.010329,0.120875,0.430450,0.414039,0.405183,-0.000372
remote_ratio,0.028731,-0.064171,1.000000,-0.236430,-0.054025,-0.028673,-0.042451,-0.046644,-0.083142,-0.071574,-0.036928
work_year2,-0.094724,0.228290,-0.236430,1.000000,0.187748,0.011127,-0.026862,0.254548,0.243475,0.234661,0.138875
experience_level2,-0.024884,0.327173,-0.054025,0.187748,1.000000,-0.032794,0.020026,0.225464,0.251325,0.250591,0.066414
employment_type2,-0.003908,-0.010329,-0.028673,0.011127,-0.032794,1.000000,-0.052231,-0.048446,-0.024582,-0.013963,-0.041001
job_title2,0.039065,0.120875,-0.042451,-0.026862,0.020026,-0.052231,1.000000,-0.057072,-0.063911,-0.073549,0.002548
salary_currency2,-0.091121,0.430450,-0.046644,0.254548,0.225464,-0.048446,-0.057072,1.000000,0.755836,0.764744,0.102738
employee_residence2,-0.087176,0.414039,-0.083142,0.243475,0.251325,-0.024582,-0.063911,0.755836,1.000000,0.945676,0.032168
company_location2,-0.072774,0.405183,-0.071574,0.234661,0.250591,-0.013963,-0.073549,0.764744,0.945676,1.000000,0.041703


In [6]:
feature = list(corrDf.loc[(abs(corrDf.salary_in_usd)>stdRatio)&(corrDf.salary_in_usd!=1)].index)
feature

['experience_level2',
 'salary_currency2',
 'employee_residence2',
 'company_location2']

In [7]:
label = ["salary_in_usd"]

In [8]:
featuredData = encodeData.loc[:,feature]
featuredData

,experience_level2,salary_currency2,employee_residence2,company_location2
0,3,7,26,25
1,2,19,75,70
2,2,19,75,70
3,3,19,11,12
4,3,19,11,12
...,...,...,...,...
3750,3,19,75,70
3751,2,19,75,70
3752,0,19,75,70
3753,0,19,75,70


In [9]:
minMaxData = featuredData.copy()

In [10]:
minMaxData.experience_level2 = minmax_scale(minMaxData.experience_level2)
minMaxData.salary_currency2 = minmax_scale(minMaxData.salary_currency2)
minMaxData.employee_residence2 = minmax_scale(minMaxData.employee_residence2)
minMaxData.company_location2 = minmax_scale(minMaxData.company_location2)
minMaxData

,experience_level2,salary_currency2,employee_residence2,company_location2
0,1.000000,0.368421,0.337662,0.352113
1,0.666667,1.000000,0.974026,0.985915
2,0.666667,1.000000,0.974026,0.985915
3,1.000000,1.000000,0.142857,0.169014
4,1.000000,1.000000,0.142857,0.169014
...,...,...,...,...
3750,1.000000,1.000000,0.974026,0.985915
3751,0.666667,1.000000,0.974026,0.985915
3752,0.000000,1.000000,0.974026,0.985915
3753,0.000000,1.000000,0.974026,0.985915


In [11]:
labelData = baseData.loc[:,label]
labelData

,salary_in_usd
0,85847
1,30000
2,25500
3,175000
4,120000
...,...
3750,412000
3751,151000
3752,105000
3753,100000


In [12]:
print(featuredData.shape)
print(labelData.shape)

(3755, 4)
(3755, 1)


In [13]:
trainDataFeat,\
testDataFeat,\
trainDataLabel,\
testDataLabel = train_test_split(minMaxData, labelData, test_size=0.3, random_state=10)

In [14]:
testIndex = testDataLabel.index

In [15]:
print(trainDataFeat.shape)
print(trainDataLabel.shape)
print(testDataFeat.shape)
print(testDataLabel.shape)

(2628, 4)
(2628, 1)
(1127, 4)
(1127, 1)


In [16]:
model_mrthod = svm.SVC(random_state=10)

In [17]:
model = model_mrthod.fit(X = trainDataFeat,
                         y = trainDataLabel)

C:\Users\caose\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
predict = pd.DataFrame(model.predict(X = testDataFeat), columns=["predict"])

In [25]:
testDataLabel.reset_index(drop=True, inplace=True)

In [26]:
resultData = pd.concat([testDataLabel,predict],axis=1)

In [27]:
testDataFeat.reset_index(drop=True, inplace=True)

In [28]:
resultData2 = pd.concat([testDataFeat,resultData],axis=1)
resultData2

,experience_level2,salary_currency2,employee_residence2,company_location2,salary_in_usd,predict
0,1.000000,1.0,0.974026,0.985915,116000,120000
1,1.000000,1.0,0.974026,0.985915,204500,120000
2,0.666667,1.0,0.974026,0.985915,220000,100000
3,1.000000,1.0,0.974026,0.985915,108000,120000
4,0.666667,1.0,0.974026,0.985915,193900,100000
...,...,...,...,...,...,...
1122,0.666667,1.0,0.766234,0.985915,45760,100000
1123,1.000000,1.0,0.974026,0.985915,81666,120000
1124,1.000000,1.0,0.974026,0.985915,136000,120000
1125,1.000000,1.0,0.974026,0.985915,150000,120000


In [31]:
feature

['experience_level2',
 'salary_currency2',
 'employee_residence2',
 'company_location2']

In [39]:
feature2 = []
for i in range(0,len(feature)):
    featCol = feature[i].split("2")
    feature2.append(featCol[0])
feature2

['experience_level',
 'salary_currency',
 'employee_residence',
 'company_location']

In [43]:
concatData = baseData.loc[testIndex,feature2]
concatData.reset_index(drop = True, inplace = True)
concatData

,experience_level,salary_currency,employee_residence,company_location
0,SE,USD,US,US
1,SE,USD,US,US
2,MI,USD,US,US
3,SE,USD,US,US
4,MI,USD,US,US
...,...,...,...,...
1122,MI,USD,PH,US
1123,SE,USD,US,US
1124,SE,USD,US,US
1125,SE,USD,US,US


In [46]:
finalData = pd.concat([concatData,resultData], axis = 1)
finalData
# salary_in_usd가 숫자가 기니까 너무 값이 많이 나오는 것 같다.
# 돈값의 범위를 지정해서 학습시키는게 어떨까?

,experience_level,salary_currency,employee_residence,company_location,salary_in_usd,predict
0,SE,USD,US,US,116000,120000
1,SE,USD,US,US,204500,120000
2,MI,USD,US,US,220000,100000
3,SE,USD,US,US,108000,120000
4,MI,USD,US,US,193900,100000
...,...,...,...,...,...,...
1122,MI,USD,PH,US,45760,100000
1123,SE,USD,US,US,81666,120000
1124,SE,USD,US,US,136000,120000
1125,SE,USD,US,US,150000,120000


In [47]:
accuracy_score(finalData.salary_in_usd, finalData.predict)

0.02040816326530612